# 1. Implement a sample query from an array with different sample conditions. 
***
## Importing the necessary libraries

In [7]:
import os
import dotenv
from bson import ObjectId

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

## Loading the environment variables


In [8]:
dotenv.load_dotenv()

True

## Creating the connection to the MongoDB


In [9]:
client = MongoClient(os.getenv("URI"), server_api=ServerApi('1'), tls=True, tlsAllowInvalidCertificates=True)

## Check connection to mongo by get standard db movies


In [10]:
def print_dict(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + str(key) + ':', end=' ')
        if isinstance(value, dict):
            print()
            print_dict(value, indent + 4)
        else:
            print(value)

In [11]:
for doc in client['sample_mflix']["movies"].find().limit(2): # You can change limit
    print_dict(doc)
    print("\n")

_id: 573a1390f29313caabcd42e8
plot: A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.
genres: ['Short', 'Western']
runtime: 11
cast: ['A.C. Abadie', "Gilbert M. 'Broncho Billy' Anderson", 'George Barnes', 'Justus D. Barnes']
poster: https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg
title: The Great Train Robbery
fullplot: Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.
languages: ['English']
released: 1903-12-01 00:00:00
directors: ['Edwin S. Porter']
rated: TV-G
awards: 
    wins: 1
    nominations: 0
    text: 1 win.
lastupdated: 2015-08-13 00:27:59.177000000
year: 1903
imdb: 
    rating: 7.4


## Creating the sample query request

In [12]:
db = client['sample_mflix']["movies"]

In [34]:
query1 = db.find(
        {
            
            "cast": {"$all": ["Tom Cruise", "Steve Harris"]}
        },
        {
            "title": 1,
            "cast": 1
        }
    
    )

In [35]:

for doc in query1.limit(5):
    print_dict(doc)
print("\n")

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Tom Cruise', 'Max von Sydow', 'Steve Harris', 'Neal McDonough']




# 2. Implement array-length sampling queries.

In [42]:
query2 = db.find(
        {
            "cast": {"$size": 3}
        },
        {
            "_id": 0,
            "title": 1,
            "cast": 1
        }
    )

In [43]:
for doc in query2.limit(5):
    print_dict(doc)

cast: ['Winsor McCay', 'George McManus', 'Roy L. McCardell']
title: Gertie the Dinosaur
cast: ['Harold Lloyd', 'Mildred Davis', 'Anna Mae Bilson']
title: Now or Never
cast: ['Charles King', 'Anita Page', 'Bessie Love']
title: The Broadway Melody
cast: ['Warner Baxter', 'Edmund Lowe', 'Dorothy Burgess']
title: In Old Arizona
cast: ['Gloria Swanson', 'Walter Byron', 'Seena Owen']
title: Queen Kelly


# 3. Implement selection requests from an array with control elements lazy sampling.

In [63]:
query3 = db.find(
        {
            "_id": ObjectId("573a139ff29313caabcff3cf"),


        },
        {
            "_id": 1,
            "title": 1,
            "cast": {"$slice": 2}
        }
    )

In [64]:
for doc in query3.limit(5):
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Tom Cruise', 'Max von Sydow']


# 4. Execute requests to add new data to the array from the using operators that check and do not check for the uniqueness of new elements.

In [98]:
def query4():
    return db.find(
        {
            "_id": ObjectId("573a139ff29313caabcff3cf"),


        },
        {
            "_id": 1,
            "title": 1,
            "cast": 1
        }
    )

In [99]:
db.update_one(
    {
        "_id": ObjectId("573a139ff29313caabcff3cf")
    },
    {
        "$push": {"cast": "Tom Cruise"}
    }
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000069'), 'opTime': {'ts': Timestamp(1726816095, 18), 't': 105}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726816095, 18), 'signature': {'hash': b'\xc5\xd6W\xfa\xbe\x04\xb5c7o-yS\x1c\xf5\x14\x9aR\r;', 'keyId': 7359210682539048965}}, 'operationTime': Timestamp(1726816095, 18), 'updatedExisting': True}, acknowledged=True)

In [100]:
for doc in query4():
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Max von Sydow', 'Steve Harris', 'Neal McDonough', 'Tom Cruise', 'Tom Cruise']


In [101]:
db.update_one(
    {
        "_id": ObjectId("573a139ff29313caabcff3cf")
    },
    {
        "$addToSet": {"cast": "Anna de'Armas"}
    }
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000069'), 'opTime': {'ts': Timestamp(1726816097, 15), 't': 105}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726816097, 15), 'signature': {'hash': b'\xf3\xb6U\x92\xba\x931\xcd\xac\xa6\xdeX\xba\x16\xd7X\xe8\x9d\xd5\xc2', 'keyId': 7359210682539048965}}, 'operationTime': Timestamp(1726816097, 15), 'updatedExisting': True}, acknowledged=True)

In [102]:
for doc in query4():
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Max von Sydow', 'Steve Harris', 'Neal McDonough', 'Tom Cruise', 'Tom Cruise', "Anna de'Armas"]


# 5. Execute a request to remove elements from the array.

In [103]:
db.update_one(
    {
        "_id": ObjectId("573a139ff29313caabcff3cf")
    },
    {
        "$pop": {"cast": -1}
    }
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000069'), 'opTime': {'ts': Timestamp(1726816104, 15), 't': 105}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726816104, 15), 'signature': {'hash': b'g4R\xdd\xfc\x1d\xcd;\x17l\xb5\x1b\xae\x81\xea*\x06\x12-\x07', 'keyId': 7359210682539048965}}, 'operationTime': Timestamp(1726816104, 15), 'updatedExisting': True}, acknowledged=True)

In [104]:
for doc in query4():
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Steve Harris', 'Neal McDonough', 'Tom Cruise', 'Tom Cruise', "Anna de'Armas"]


In [105]:
db.update_one(
    {
        "_id": ObjectId("573a139ff29313caabcff3cf")
    },
    {
        "$pull": {"cast": 'Tom Cruise'}
    }
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000069'), 'opTime': {'ts': Timestamp(1726816150, 12), 't': 105}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726816150, 12), 'signature': {'hash': b'\x81s\x9d?;\x94;\xe4^\xc36\xff\xa5nt\xd8l\xe7\x06l', 'keyId': 7359210682539048965}}, 'operationTime': Timestamp(1726816150, 12), 'updatedExisting': True}, acknowledged=True)

In [106]:
for doc in query4():
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Steve Harris', 'Neal McDonough', "Anna de'Armas"]


# 6. Execute the request to change the values of the array.

In [110]:
db.update_one(
    {
        "_id": ObjectId("573a139ff29313caabcff3cf"),
    },
    {
        "$set": {"cast.0": "Tom Cruise"}
    }
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000069'), 'opTime': {'ts': Timestamp(1726816632, 9), 't': 105}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726816632, 10), 'signature': {'hash': b'\xc9\x97a\xb0z\xf0\xf1t\xcc\xac\x9bjb\xa7\xf30J\xa8*X', 'keyId': 7359210682539048965}}, 'operationTime': Timestamp(1726816632, 9), 'updatedExisting': True}, acknowledged=True)

In [111]:
for doc in query4():
    print_dict(doc)

_id: 573a139ff29313caabcff3cf
title: Minority Report
cast: ['Tom Cruise', 'Neal McDonough', "Anna de'Armas"]


# 7. Implement two queries using regular expressions.
***
## Creating the first sample query request

In [113]:
query5 = db.find(
        {
            "title": {"$regex": ".*The.*"}
        },
        {
            "_id": 0,
            "title": 1,
            "cast": 1
        }
    )

In [114]:
for doc in query5.limit(5):
    print_dict(doc)

cast: ['A.C. Abadie', "Gilbert M. 'Broncho Billy' Anderson", 'George Barnes', 'Justus D. Barnes']
title: The Great Train Robbery
cast: ['Pearl White', 'Crane Wilbur', 'Paul Panzer', 'Edward Josè']
title: The Perils of Pauline
cast: ['George Beban', 'Clara Williams', 'J. Frank Burke', 'Leo Willis']
title: The Italian
cast: ['Mary Pickford', 'Madlaine Traverse', 'Charles Wellesley', 'Gladys Fairbanks']
title: The Poor Little Rich Girl
cast: ['Tula Belle', 'Robin Macdougall', 'Edwin E. Reed', 'Emma Lowry']
title: The Blue Bird


## Creating the second sample query request

In [126]:
query6 = db.find(
        {
            "fullplot": {"$regex": ".*American.*"}
        },
        {
            "_id": 0,
            "title": 1,
            "fullplot": 1
        }
    )

In [127]:
for doc in query6.limit(5):
    print_dict(doc)

title: The Great Train Robbery
fullplot: Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.
title: The Strong Man
fullplot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by long distance. After the war, the young Belgian journeys to America as assistant to a theatrical "strong man", Zandow the Great (Arthur Thalasso). While in America, he searches for Mary Brown... and he finds her, just as word comes that Zandow is incapacitated and the little nebbish must go on stage in his place.
title: Four Sons
fullplot: In Burgendorf, Bavaria, Mother Bernle has four sons. Franz is in the army, Johann works at the 